In [1]:
#Nate's PageRank algorithm

import numpy as np
from scipy import linalg as la
import networkx as nx
import itertools

Class for representing directed graphs via their adjacency matrices...
- We calculate $\hat{A}$ as follows: $$\hat{A}_{ij} = \frac{\tilde{A}_{ij}}{\sum_{k=1} \tilde{A}_{kj}} $$
where $\tilde{A}$ is $A$ (the adjacency matrix) with all "sinks" (columns of zeros) replaced as columns of ones.
- Then we find the PageRank vector $\textbf{p}$ in three different ways:
- We solve the linear system $$(I-\epsilon\hat{A})\textbf{p}=\frac{1-\epsilon}{n}\textbf{1}$$
- We let $E$ be a matrix of ones and define $B=\epsilon\hat{A}+\frac{1-\epsilon}{n}E$. Then $\textbf{p}$ is an eigenvector of $B$ corresponding to the largest eigenvalue, $\lambda=1$.
- We pick an initial $\textbf{p}(0)$ and iteratively solve with $$\textbf{p}(t+1) = \epsilon\hat{A}\textbf{p}(t) + \frac{1-\epsilon}{n}\textbf{1}$$

In [2]:
class DiGraph:
    """A class for representing directed graphs via their adjacency matrices.

    Attributes:
        (fill this out after completing DiGraph.__init__().)
    """
    
    def __init__(self, A, labels=None):
        """Modify A so that there are no sinks in the corresponding graph. 
        (Sinks are columns of zeros, representing pages that don't link anywhere.)
        Then calculate Ahat. Save Ahat and the labels as attributes.

        Parameters:
            A ((n,n) ndarray): the adjacency matrix of a directed graph.
                A[i,j] is the weight of the edge from node j to node i.
            labels (list(str)): labels for the n nodes in the graph.
                If None, defaults to [0, 1, ..., n-1].
        """
        #Check for zeros to see if we need to change them
        num_rows,num_cols = A.shape
        for i in range(num_cols):
            if np.allclose(A[:,i], np.zeros(num_rows)):
                A[:,i] = np.ones(num_rows)
        #Now A is A_tilda
        #We apply the above equation to make A_hat
        big_sum_vector = np.array([sum(A[k,j] for k in range(num_rows)) for j in range(num_cols)])
        A = A / big_sum_vector
        #Now A is A_hat, so we store it as an attribute
        self.A_hat = A

        #Now what are the labels?
        if labels is None:
            labels = np.array(list(range(num_cols)))
        else:
            if len(labels) != num_cols:
                raise ValueError("Number of labels must equal number of nodes.")

        #Store this information as attributes
        self.labels = labels
        self.num_nodes = num_cols

    #    We will solve for the PageRank vector p three different ways...
    
    def linsolve(self, epsilon=0.85):
        """Compute the PageRank vector using the linear system method.

        Parameters:
            epsilon (float): the damping factor, between 0 and 1.

        Returns:
            dict(str -> float): A dictionary mapping labels to PageRank values.
        """
        #Get p
        p = la.solve((np.eye(self.num_nodes) - epsilon*self.A_hat), ((1-epsilon)/self.num_nodes) * np.ones(self.num_nodes) )
        #Normalize p
        p = p/la.norm(p,ord=1)
        #Get the dictionary mapping labels to the PageRank
        keys = list(self.labels)
        values = list(p)
        dictionarp = dict(zip(keys, values))

        return dictionarp

    def eigensolve(self, epsilon=0.85):
        """Compute the PageRank vector using the eigenvalue method.
        Normalize the resulting eigenvector so its entries sum to 1.

        Parameters:
            epsilon (float): the damping factor, between 0 and 1.

        Return:
            dict(str -> float): A dictionary mapping labels to PageRank values.
        """
        #Get B
        B = epsilon*self.A_hat + ((1-epsilon)/self.num_nodes)*np.ones((self.num_nodes,self.num_nodes))
        #Find first eignenvector of B
        vals,vecs = la.eig(B)[0],la.eig(B)[1]
        p = vecs[:,np.argmax(vals)]
        #Scale it!
        p = p / la.norm(p,ord=1)

        #Get the dictionary mapping labels to the PageRank
        keys = list(self.labels)
        values = list(p)
        dictionarp = dict(zip(keys, values))

        return dictionarp

    def itersolve(self, epsilon=0.85, maxiter=100, tol=1e-12):
        """Compute the PageRank vector using the iterative method.

        Parameters:
            epsilon (float): the damping factor, between 0 and 1.
            maxiter (int): the maximum number of iterations to compute.
            tol (float): the convergence tolerance.

        Return:
            dict(str -> float): A dictionary mapping labels to PageRank values.
        """
        #First get p0
        p0 = (1/self.num_nodes)*np.ones(self.num_nodes)
        one_vec = np.ones(self.num_nodes)

        #Now we iterate
        for t in range(maxiter):
            #Get p1
            p1 = epsilon*self.A_hat@p0 + ((1-epsilon)/self.num_nodes)*one_vec
            #Check if we need to STOP!
            if la.norm(p1-p0,ord=1) < tol:
                break
            #Now start over
            p0 = p1

        #Get the dictionary mapping labels to the PageRank
        keys = list(self.labels)
        values = list(p1)
        dictionarp = dict(zip(keys, values))

        return dictionarp

Check the results to make sure they're the same.

In [3]:
#Let's test...
A = np.array([[0,0,0,0],[1,0,1,0],[1,0,0,1],[1,0,1,0]])
epsilon = .85
DG = DiGraph(A)

#Solve
print("Linearly solving:",str(DG.linsolve(epsilon)))
print("Eigensolving:",str(DG.eigensolve(epsilon)))
print("Iterative method:",str(DG.itersolve(epsilon)))
print("They are the same!")

Linearly solving: {0: 0.09575863576738085, 1: 0.27415828596414515, 2: 0.35592479230432883, 3: 0.27415828596414515}
Eigensolving: {0: 0.09575863576738075, 1: 0.2741582859641452, 2: 0.3559247923043289, 3: 0.2741582859641452}
Iterative method: {0: 0.09575863576740319, 1: 0.27415828596408354, 2: 0.35592479230442975, 3: 0.27415828596408354}
They are the same!


This next funciton uses the code I wrote above to return a ranked list of the labels.

In [4]:
def get_ranks(d):
    """Construct a sorted list of labels based on the PageRank vector.

    Parameters:
        d (dict(str -> float)): a dictionary mapping labels to PageRank values.

    Returns:
        (list) the keys of d, sorted by PageRank value from greatest to least.
    """
    #Get lists
    labels = list(d.keys())
    ranks = list(d.values())
    #Now get the order
    ord = list(np.argsort(ranks))[::-1]
    #Sort that new list
    labels = np.array(labels)
    sorted_labels = labels[ord]
    return list(sorted_labels)

In [5]:
#Example from above continued...
get_ranks(DG.itersolve(epsilon))

[2, 3, 1, 0]

The file web_stanford.txt contains information on Stanford University webpages and the hyperlinks between them, gathered in 2002. This function gives us a ranked list of the webpages. (Note the labels in the file are webpage IDs, so they'll be nasty-looking numbers.)

In [6]:
def rank_websites(filename="web_stanford.txt", epsilon=0.85):
    """Read the specified file and construct a graph where node j points to
    node i if webpage j has a hyperlink to webpage i. Use the DiGraph class
    and its itersolve() method to compute the PageRank values of the webpages,
    then rank them with get_ranks().

    Each line of the file has the format
        a/b/c/d/e/f...
    meaning the webpage with ID 'a' has hyperlinks to the webpages with IDs
    'b', 'c', 'd', and so on.

    Parameters:
        filename (str): the file to read from.
        epsilon (float): the damping factor, between 0 and 1.

    Returns:
        (list(str)): The ranked list of webpage IDs.
    """
    #read in the data
    with open(filename,'r') as myfile:
        data = myfile.readlines()
    #data is a list of strings
    for i in range(len(data)):
        data[i] = data[i].strip().split('/')
    #Now data is a list of lists, lol
    unique_nodes = set(item[i] for item in data for i in range(len(item)))
    #Make an alphabetical list
    ordered_nodes = list(unique_nodes)
    ordered_nodes.sort()
    #make a dictionary
    num = len(ordered_nodes)
    nodes_to_numbers = dict(zip(ordered_nodes, list(range(num))))
    #Make a blank matrix
    M = np.zeros((num, num))
    #Now we do a CRAZY LOOP
    for i in range(len(data)):
        #What row are we looking at
        row_in_question = data[i]
        #Save number of first entry
        i_matrix = nodes_to_numbers[row_in_question[0]]
        for j in range(1,len(row_in_question)):
            datum_in_question = row_in_question[j]
            #Save number of entry
            j_matrix = nodes_to_numbers[datum_in_question]
            #Update matrix
            M[j_matrix,i_matrix] = 1
    #call the class!
    DGM = DiGraph(M,labels=ordered_nodes)
    ranks_dict = DGM.itersolve(epsilon=epsilon)

    return get_ranks(ranks_dict)

In [7]:
#Do it!
rank_websites()

['98595',
 '32791',
 '28392',
 '77323',
 '92715',
 '26083',
 '130094',
 '99464',
 '12846',
 '106064',
 '332',
 '31328',
 '86049',
 '123900',
 '74923',
 '90571',
 '119538',
 '116900',
 '139197',
 '15672',
 '114623',
 '20283',
 '136623',
 '108608',
 '6213',
 '56800',
 '62259',
 '67827',
 '64104',
 '96254',
 '82752',
 '203109',
 '178606',
 '24083',
 '217557',
 '68912',
 '41471',
 '203100',
 '19894',
 '177473',
 '203696',
 '121480',
 '268900',
 '246911',
 '146603',
 '10984',
 '84478',
 '210258',
 '182230',
 '147255',
 '110923',
 '185201',
 '102329',
 '178003',
 '247003',
 '208254',
 '188707',
 '197440',
 '228036',
 '199855',
 '230247',
 '110520',
 '121418',
 '112786',
 '101455',
 '41677',
 '66498',
 '88621',
 '84094',
 '63712',
 '35759',
 '61413',
 '55398',
 '17101',
 '32789',
 '51075',
 '21554',
 '72982',
 '1662',
 '104202',
 '94657',
 '4889',
 '48096',
 '81525',
 '11433',
 '26918',
 '79322',
 '237162',
 '132956',
 '79383',
 '264536',
 '31597',
 '76236',
 '209406',
 '215281',
 '253851',
 

I'll use this same strategy to rank basketball teams in a given year.
Files to use:
- ncaa2014.csv
- ncaa2015.csv
- ncaa2016.csv
- ncaa2017.csv

In [8]:
def rank_ncaa_teams(filename, epsilon=0.85):
    """Read the specified file and construct a graph where node j points to
    node i with weight w if team j was defeated by team i in w games. Use the
    DiGraph class and its itersolve() method to compute the PageRank values of
    the teams, then rank them with get_ranks().

    Each line of the file has the format
        A,B
    meaning team A defeated team B.

    Parameters:
        filename (str): the name of the data file to read.
        epsilon (float): the damping factor, between 0 and 1.

    Returns:
        (list(str)): The ranked list of team names.
    """
    #read in the data
    with open(filename,'r') as myfile:
        data = myfile.readlines()
    #data is a list of strings
    for i in range(len(data)):
        data[i] = data[i].strip().split(',')
    #Now data is a list of lists, lol
    #Get rid of the header
    data = data[1:]
    #We want a list of unique teams
    unique_teams_set = set(item[i] for item in data for i in range(2))
    unique_teams_list = list(unique_teams_set)
    unique_teams_list.sort()
    num_teams = len(unique_teams_list)
    #And we want a dictionary associating each team with a number
    teams_to_numbers = dict(zip(unique_teams_list, list(range(num_teams))))

    #   Now we make the matrix representing a graph representing the wins and loses
    #   Losers point to winners, path weight corresponds to number of losses
    #   E.g. If team A lost to team B twice, we have A--(2)-->B
    #   ...and the B,A entry of the matrix is equal to 2

    M = np.zeros((num_teams,num_teams))
    for i in range(len(data)):
        winner_num = teams_to_numbers[data[i][0]]
        loser_num = teams_to_numbers[data[i][1]]
        M[winner_num,loser_num] += 1
    #Now M is constructed
    DGM = DiGraph(M,labels=unique_teams_list)
    ranks_dict = DGM.itersolve(epsilon=epsilon)

    return get_ranks(ranks_dict)

Now let's have some fun. :)

In [9]:
rank_ncaa_teams('ncaa2014.csv')

['Duke',
 'Wisconsin',
 'Notre Dame',
 'North Carolina State',
 'Virginia',
 'Kentucky',
 'Maryland',
 'Louisville',
 'Michigan State',
 'UNC',
 'Miami (FL)',
 'Arizona',
 'Villanova',
 'Kansas',
 'Providence',
 'Iowa State',
 'West Virginia',
 'Baylor',
 'Xavier',
 'Oklahoma',
 'Georgetown',
 'Pitt',
 'Utah',
 'Syracuse',
 'Rutgers',
 'Wichita State',
 'VCU',
 'Iowa',
 'Gonzaga',
 'Northern Iowa',
 'San Diego State',
 'Oregon',
 'Butler',
 'Ohio State',
 'Indiana',
 'Stanford',
 'Arizona State',
 'Seton Hall',
 'Cincinnati',
 'Purdue',
 'Illinois',
 "St. John's (NY)",
 'Arkansas',
 'Dayton',
 'Kansas State',
 'UCLA',
 'UNLV',
 'Oregon State',
 'BYU',
 'Wyoming',
 'Oklahoma State',
 'Illinois State',
 'Old Dominion',
 'SMU',
 'LSU',
 'Boise State',
 'Colorado State',
 'Temple',
 'Ole Miss',
 'Texas',
 'Valparaiso',
 'Richmond',
 'Clemson',
 'Washington',
 'George Washington',
 'Minnesota',
 'Green Bay',
 'Alabama-Birmingham',
 'Davidson',
 'Evansville',
 'Michigan',
 'Georgia',
 'Georg

In [10]:
rank_ncaa_teams('ncaa2015.csv')

['Villanova',
 'Virginia',
 'Oklahoma',
 'Kansas',
 'UNC',
 'Oregon',
 'Xavier',
 'Seton Hall',
 'Michigan State',
 'Miami (FL)',
 'West Virginia',
 'Iowa State',
 'Duke',
 'Texas',
 'Wisconsin',
 'Notre Dame',
 'Utah',
 'Syracuse',
 'Iowa',
 'Indiana',
 'Providence',
 'Texas A&M',
 'George Washington',
 'Kentucky',
 'University of California',
 'Purdue',
 'Northern Iowa',
 'Louisville',
 'Texas Tech',
 'Maryland',
 "St. Joseph's",
 'Georgia Tech',
 'Virginia Tech',
 'Arizona',
 'Baylor',
 'Gonzaga',
 'Colorado',
 'Oregon State',
 'Butler',
 'San Diego State',
 'Dayton',
 'Pitt',
 'Wichita State',
 'VCU',
 'Florida State',
 'UConn',
 'Michigan',
 'Valparaiso',
 'Florida',
 'BYU',
 'Creighton',
 "Saint Mary's (CA)",
 'Stanford',
 'SMU',
 'South Carolina',
 'Clemson',
 'Boise State',
 'St. Bonaventure',
 'Long Beach State',
 'USC',
 'Cincinnati',
 'Alabama',
 'Georgia',
 'Fresno State',
 'UCLA',
 'Little Rock',
 'UNLV',
 'Vanderbilt',
 'Georgetown',
 'Middle Tennessee',
 'Tulsa',
 'LSU',

In [11]:
rank_ncaa_teams('ncaa2016.csv')

['Georgetown',
 'Illinois',
 'Xavier',
 'DePaul',
 "St. John's (NY)",
 'Creighton',
 'Maryland',
 'Portland',
 'Louisville',
 'Pittsburgh',
 'Marquette',
 'UCLA',
 'Florida St.',
 'Providence',
 'Villanova',
 'Seton Hall',
 'Arizona',
 'Northwestern',
 'Minnesota',
 'Butler',
 'Syracuse',
 'Ohio St.',
 'Notre Dame',
 'Michigan St.',
 'Stanford',
 'Iowa',
 'Purdue',
 'North Carolina',
 'Colorado',
 'Wisconsin',
 'Michigan',
 'Nebraska',
 'Duke',
 'Southern California',
 'Arizona St.',
 'Oregon',
 'Virginia',
 'Tennessee',
 'UNLV',
 'Georgia Tech',
 'Washington',
 'Oklahoma',
 'Kentucky',
 'Indiana',
 'Miami (FL)',
 'Kansas',
 'Texas',
 'Wake Forest',
 'Boston College',
 'California',
 'Washington St.',
 'Utah',
 'Penn St.',
 'Iowa St.',
 'Oregon St.',
 'Ole Miss',
 'TCU',
 'Texas Tech',
 'Clemson',
 'UNI',
 'Baylor',
 'BYU',
 'Utah St.',
 'NC State',
 'Pacific',
 'Georgia',
 'South Carolina',
 'West Virginia',
 'San Francisco',
 'Kansas St.',
 'Missouri',
 'Virginia Tech',
 'Loyola Mary

In [12]:
rank_ncaa_teams('ncaa2017.csv')

['Villanova',
 'Kansas',
 'Providence',
 'West Virginia',
 'Michigan',
 'Texas Tech',
 'Virginia',
 'UNC',
 'Xavier',
 'Butler',
 'Florida',
 'Duke',
 'Kentucky',
 'Tennessee',
 'Purdue',
 'Texas A&M',
 'Creighton',
 'Alabama',
 'Oklahoma State',
 'Florida State',
 'Virginia Tech',
 'Arizona',
 'Clemson',
 'Auburn',
 "St. John's (NY)",
 'Oklahoma',
 'Loyola (IL)',
 'Kansas State',
 'Arkansas',
 'Nevada',
 'Arizona State',
 'Seton Hall',
 'Cincinnati',
 'Houston',
 'Ohio State',
 'Texas Christian',
 'Gonzaga',
 'Wichita State',
 'Missouri',
 'Baylor',
 'Syracuse',
 'Washington',
 'LSU',
 'North Carolina State',
 'Texas',
 'Utah',
 'Georgia',
 'Western Kentucky',
 'Mississippi State',
 'Michigan State',
 'Penn State',
 'USC',
 'Marquette',
 'UCLA',
 'Oregon',
 'San Diego State',
 'Miami (FL)',
 'UMBC',
 'Colorado',
 'Louisville',
 'Stanford',
 "Saint Mary's (CA)",
 'South Carolina',
 'Rhode Island',
 'Notre Dame',
 'Temple',
 'St. Bonaventure',
 'Middle Tennessee',
 'Boise State',
 'Wyom

Now we rank actors. The directed graph here has nodes for actors linked to the node for the lead actor in any movie they were in. (E.g. Michael Cain $\rightarrow$ Christian Bale in The Dark Knight Rises.) The most linked-to actors appear first in the ranked list. Note that the ranked list may be different with a different damping factor $\epsilon$.

In [13]:
def rank_actors(filename="top250movies.txt", epsilon=0.85):
    """Read the specified file and construct a graph where node a points to
    node b with weight w if actor a and actor b were in w movies together but
    actor b was listed first. Use NetworkX to compute the PageRank values of
    the actors, then rank them with get_ranks().

    Each line of the file has the format
        title/actor1/actor2/actor3/...
    meaning actor2 and actor3 should each have an edge pointing to actor1,
    and actor3 should have an edge pointing to actor2.
    """

    #   Read in the data and turn it into a list of lists of actor names

    #read in the data
    with open(filename,'r',encoding="utf-8") as myfile:
        data = myfile.readlines()
    #data is a list of strings
    for i in range(len(data)):
        data[i] = data[i].strip().split('/')[1:]
    #Now data is a list of lists without movie titles

    #   Now we convert each list in data into a list of combinations and
    #   feed those into the DG (directed graph).
    #   We want a graph with secondary actors pointing to primary actors with a
    #   weight of how many movies they were in together (in that same order).

    DG = nx.DiGraph()
    for row in range(len(data)):
        movie_in_question = data[row]
        actor_combos = list(itertools.combinations(movie_in_question,2))
        #^Note that actor_combos is a list of tuples, with the primary actor
        #first in each tuple
        for combo in actor_combos:
            #Check if the edge is already there
            if DG.has_edge(combo[1],combo[0]):
                DG[combo[1]][combo[0]]["weight"] += 1
            else:
                DG.add_edge(combo[1],combo[0],weight=1)

    #   The DG should be all constructed now
    #   We need to get the pagerank dictionary and then get a ranked list of
    #   actors from the function from Problem 3

    pr_dict = nx.pagerank(DG,alpha=epsilon)
    return get_ranks(pr_dict)

Let's try it out!

In [14]:
rank_actors()

['Leonardo DiCaprio',
 'Jamie Foxx',
 'Christoph Waltz',
 'Robert De Niro',
 'Al Pacino',
 'Tom Hanks',
 'Antonella Attili',
 'Brad Pitt',
 'Christian Bale',
 'Ben Kingsley',
 'Diahnne Abbott',
 'Liam Neeson',
 'Ralph Fiennes',
 'Matt Damon',
 'Tom Hardy',
 'Gary Oldman',
 'Morgan Freeman',
 'Karen Gillan',
 'Ryan Gosling',
 'Jack Nicholson',
 'Harrison Ford',
 'Michael Caine',
 'Kevin Spacey',
 'Mark Ruffalo',
 'Enzo Cannavale',
 'Sumi Shimamoto',
 'Joseph Gordon-Levitt',
 'James Stewart',
 'Thomas Mitchell',
 'Marlon Brando',
 'Hugh Jackman',
 'Zoe Saldana',
 'Frank Adu',
 'Harvey Keitel',
 'Clint Eastwood',
 'Daniel Day-Lewis',
 'Bruce Willis',
 'Eli Wallach',
 'F. Murray Abraham',
 'Mélanie Laurent',
 'Tim Roth',
 'Faye Dunaway',
 'Ellen Burstyn',
 'Robert Duvall',
 'Matthew McConaughey',
 'Mark Hamill',
 'Ben Affleck',
 'Chieko Baishô',
 'Isa Danieli',
 'Charles Chaplin',
 'Chris Hemsworth',
 'Jonah Hill',
 'Josh Brolin',
 'Dave Bautista',
 'Chiwetel Ejiofor',
 'Michael Berryman',

In [15]:
rank_actors(epsilon=.7)

['Leonardo DiCaprio',
 'Robert De Niro',
 'Tom Hanks',
 'Jamie Foxx',
 'Al Pacino',
 'Christoph Waltz',
 'Christian Bale',
 'Ben Kingsley',
 'Ralph Fiennes',
 'Matt Damon',
 'Morgan Freeman',
 'Tom Hardy',
 'Liam Neeson',
 'Brad Pitt',
 'Gary Oldman',
 'Harrison Ford',
 'Ryan Gosling',
 'Kevin Spacey',
 'James Stewart',
 'Clint Eastwood',
 'Diahnne Abbott',
 'Michael Caine',
 'Antonella Attili',
 'Jack Nicholson',
 'Thomas Mitchell',
 'Hugh Jackman',
 'F. Murray Abraham',
 'Mark Ruffalo',
 'Eli Wallach',
 'Joseph Gordon-Levitt',
 'Charles Chaplin',
 'Bruce Willis',
 'Karen Gillan',
 'Marlon Brando',
 'Mark Hamill',
 'Robert Duvall',
 'Sumi Shimamoto',
 'Faye Dunaway',
 'Aamir Khan',
 'Ben Affleck',
 'Martin Sheen',
 'Joe Pesci',
 'Matthew McConaughey',
 'Daniel Day-Lewis',
 'Zoe Saldana',
 'Harvey Keitel',
 'Darsheel Safary',
 'Chris Hemsworth',
 'Jonah Hill',
 'William Holden',
 'Emma Stone',
 'Tim Roth',
 'Frank Adu',
 'Enzo Cannavale',
 'Ellen Burstyn',
 'Ryûnosuke Kamiki',
 'Alec G